In [1]:
import shutil
from osgeo import gdal, osr
from PIL import Image
import numpy as np
from GPSPhoto import gpsphoto
import utm
import glob
import math

In [2]:
#Carpeta de trabajo
path = '/SKETCH_DRONE_PROJECT/CARRO_MOTO_TARDE'
##path = '/georef/CARRO_MOTO_TARDE'

In [3]:
#Función para cálcular el ancho y alto que cubre la imagen en metros 

def huella(s, hv, df, width, high):
    
    #Cálculo de gsd
    gsd = (s * hv * 100) / (df * width)

    #s = Ancho del sensor [mm]
    #hv = Altura de vuelo en [m]
    #df = Distancia Focal cámara [m]
    #width = Ancho de la imagen [pixeles]
    
    #Alto y ancho de la imagen en metros
    ly = ((gsd * high) / 100) / 1000
    #hight = alto de la imagen [pixeles]
    lx = ((gsd * width) / 100) / 1000
    
    return(lx, ly)

#Función para generar una matríz con las coordenadas de los puntos de control
def coordinates_gcp(lat_c,lon_c,lx,ly):
    
    coordinates = np.zeros((5,2)) 

    coordinates[0,:] = lat_c, lon_c #Arreglo que guardará las coordenadas de la imagen


    utmx = utm.from_latlon(lat_c,lon_c, 18, 'N')[0]
    utmy = utm.from_latlon(lat_c,lon_c, 18, 'N')[1]
    
    ## Coordenadas de las esquina superior izquierda
    x0 = utmx - (lx/2.0)
    y0 = utmy - (ly/2.0)
    
    coordinates[1,:] = ([utm.to_latlon(x0, y0, 18, 'N')[0],utm.to_latlon(x0, y0, 18, 'N')[1]])
    
    ## Coordenadas de las esquina inferior izquierda
    x1 = utmx - (lx/2.0)
    y1 = utmy + (ly/2.0)
    
    coordinates[2,:] = ([utm.to_latlon(x1, y1, 18, 'N')[0],utm.to_latlon(x1, y1, 18, 'N')[1]])
    
    ## Coordenadas de las esquina superior derecha
    
    x2 = utmx + (lx/2.0)
    y2 = utmy - (ly/2.0)
    
    coordinates[3,:] = ([utm.to_latlon(x2, y2, 18, 'N')[0],utm.to_latlon(x2, y2, 18, 'N')[1]])
    
    ## Coordenadas de las esquina inferior derecha
    
    x3 = utmx + (lx/2.0)
    y3 = utmy + (ly/2.0)
    
    coordinates[4,:] = ([utm.to_latlon(x3, y3, 18, 'N')[0],utm.to_latlon(x3, y3, 18, 'N')[1]])
    
    return(coordinates)
    # Open the output file for writing for writing:

In [4]:
#Lista de nombres de todas las imagenes JPG
images = glob.glob(path+'/*.JPG')

#Ancho del sensor para la cámara
s= 6.30

#Distancia focal de la camara
df = 4.7 * 10**-3

#Iterador para nombrar las imagenes de salidas
i= 0

#For que recorre todos los nombres de la lista de imagenes JPG
for orig_fn in images:
    
    #Nombre de archivo de salida tipo tif
    ##output_fn =path + '/geopath/geo_%d' % i + ".tif"
    output_fn =path +'/SKETCH_DRONE_PROJECT/CARRO_MOTO_TARDE/geopath2/geo_%d' % i + ".tif"
    
    #Leer la imagen con la libreria Pillow 
    image = Image.open(orig_fn)
    
    #Convertir la imagen como arreglo
    imar = np.asarray(image)
    
    #Extraer alto y ancho de la imagen
    high = imar.shape[0]
    width = imar.shape[1]
    
    # Información del gps
    data = gpsphoto.getGPSData(orig_fn)
    
    #Valores de latitud, longitud y altitud del gps de la foto
    lat_c = data['Latitude']
    lon_c = data['Longitude']
    alt_c = data['Altitude']
    
    #Altura de vuelo = altitude gps - altura de Popayán sobre el nivel del mar
    hv = alt_c-1760
    
    #Alto y ancho de la imagen en metros con la función huella
    ##El orden de los parametros es:
    #huella(s=Ancho del sensor [mm],
    #hv = Altura de vuelo en [m],
    #df = Distancia Focal cámara [m],
    #width = Ancho de la imagen [pixeles]
    #high = Alto de la imagen [pixeles]):
    
    lx,ly = huella(s, hv, df, width, high)
    
    #Coordinadas de los puntos de control llamando a la función coordinates
    coordinates = coordinates_gcp(lat_c,lon_c,lx,ly)
       
        
    # Open the output file for writing for writing:

    src_ds = gdal.Open(orig_fn)
    print(src_ds)

    format = "GTiff"
    driver = gdal.GetDriverByName(format)    #formato para la imagen


    ds = driver.CreateCopy(output_fn, src_ds, 0)  #crear una copia de la imagen de entrada en formato tif


    # Set spatial reference:


    sr = osr.SpatialReference()

    sr.ImportFromEPSG(4326)  #asigna el sistema de coordenadas #4326 

    dest_wkt = sr.ExportToWkt()



    # Puntos de control 
    # Formato : [Longitude, latitude, elevación, (indice de la columna del pixel), (indice de la fila del pixel)]

    gcps =  [gdal.GCP(coordinates[1,1], coordinates[1,0] , 0, 0,0),     #Punto de control de la esquina superior izq. de la imagen 
    gdal.GCP(coordinates[2,1], coordinates[2,0] ,0,0,high+1),             #Punto de control de la esquina inferior izq. de la imagen
    gdal.GCP(coordinates[3,1], coordinates[3,0] ,width+1,high+1),           #Punto de control de la esquina superior dere. de la imagen
    gdal.GCP(coordinates[4,1], coordinates[4,0] ,0,width+1,0 )]           #Punto de control de la esquina inferior dere. de la imagen


    ds.SetProjection(sr.ExportToWkt())
    ds.SetGeoTransform(gdal.GCPsToGeoTransform(gcps))

    # Close the output file in order to be able to work with it in other programs:
    ds = None

    i +=1
    